In [1]:
import json
import os
import pandas as pd
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

from tqdm.notebook import  tqdm

with open('config.txt', 'r') as f:
    names = f.read().split('\n')

def process_roster(path):
    roster = pd.read_csv(path)
    roster['Game'] = roster.Game.str.split('@').apply(lambda x: set([y.strip() for y in x]))
    roster['Name'] = roster.Name
    roster['TeamAbbrev'] = roster.TeamAbbrev.apply(lambda x: set([x]))
    roster['OppAbbrev'] = roster.Game - roster.TeamAbbrev
    roster['Team'] = roster.TeamAbbrev.apply(lambda x: list(x)[0])
    roster['Opp'] = roster.OppAbbrev.apply(lambda x: list(x)[0])
    roster.Time = pd.to_datetime(roster.Time)
    roster.drop(['TeamAbbrev','OppAbbrev', 'Game'], axis = 1, inplace=True)
    return roster[['Name','Position', 'ID', 'Time', 'Salary', 'Team', 'Opp']]

import requests

def url_join(*args):
    return '/'.join(args)

class draftProcessor:
    def __init__(self, draft, load_ids = True):
        self.local_dir = 'data/NBA_API'
        self.base_url = "https://api-nba-v1.p.rapidapi.com"
        self.headers = {
            'x-rapidapi-host': "api-nba-v1.p.rapidapi.com",
            'x-rapidapi-key': "3c3966d785msh9c9c037d21ff8a9p1af469jsn475bf58bcbe1"
        }

        self.load_ids = load_ids
        self.draft = draft
        self.id_mapping = json.load(open(url_join(self.local_dir, 'id_mapping.json'), 'r'))

        if self.load_ids:
            self.get_player_ids()

        self.draft.index = [x.lower() for x in self.draft.index]
        self.draft = self.draft.join(pd.DataFrame.from_dict(self.id_mapping, orient='index').rename({0: 'api_id'}, axis = 1))
        self.draft.dropna(inplace=True)
        self.draft['api_id'] = self.draft['api_id'].astype(int)


    def get_player_ids(self):
        names = list(self.draft.index)

        for name in names:
            if name.lower() not in self.id_mapping:
                last_name = name.split(' ')[-1]
                print(f'id not found for {name}, getting id for {last_name}')
                try:
                    players = self.get_players({'name': last_name})
                except Exception as e:
                    print(f'failed. Error Message: {e}')
                    continue

                if type(players) == dict:
                    player_name = players['firstname'] + ' ' + players['lastname']
                    self.id_mapping[players[player_name.lower()]] = players['id']
                else:
                    for player in players:
                        player_name = player['firstname'] + ' ' + player['lastname']
                        player_id = player['id']
                        self.id_mapping[player_name.lower()] = player_id

                with open(url_join(self.local_dir, 'id_mapping.json'), 'w') as f:
                    json.dump(self.id_mapping,f)
            else:
                continue

    def get_players(self, params):
        url = url_join(self.base_url, 'players')
        print(url)
        resp= requests.get(url, headers = self.headers, params = params).json()
        return resp['response']


    def load_player_stats(self, params, ret = False):
        url = url_join(self.base_url, 'players','statistics')
        data_path = url_join(self.local_dir, 'stats', str(params['season']))
        params['id'] = int(params['id'])

        if f"{params['id']}.csv" in os.listdir(data_path):
            if ret:
                data = pd.read_csv(url_join(data_path, f"{params['id']}.csv"))
                data['id'] = params['id']
            else:
                return
        else:
            print(f"downloading data for id: {params['id']}")
            try:
                data = requests.get(url, headers=headers, params = params).json()
                data = pd.DataFrame(data['response']).drop(['team', 'player', 'game'], axis=1)
                data.to_csv(url_join(data_path, f"{params['id']}.csv"), index = False)
            except:
                print(f"failed. n_games = {len(data)}")
                return

        if ret:
            return data


In [12]:
get_contests('NBA SINGLE ENTRY $10 DOUBLE UP')

NameError: name 'Client' is not defined

In [11]:
def get_contests(names, date=dt.datetime.now()):
    contests = Client().contests(sport=Sport.NBA).contests
    names = [x.lower() for x in names]
    cinfo = {}
    for idx, contest in enumerate(contests):
        # contest_start =
        # contest_start.replace(tzinfo = et)
        if contest.name.lower() in names and contest.starts_at.replace(tzinfo=et) >= date.replace(tzinfo=et):
            cinfo[contest.name] = {
                'start_time': contest.starts_at,
                'id': contest.contest_id,
                'draft_id': contest.draft_group_id,
                'payout': contest.payout
            }
    return cinfo

In [3]:
name = 'NBA $3K AND-ONE [20 ENTRY MAX]'
date = dt.datetime.now().strftime('%Y-%m-%d')
path = f'data/{name}/{date}.csv'

df = process_roster(path).set_index('Name')
df.head()


test = draftProcessor(df, False)

In [4]:
import numpy as np
df_factors = []

def calc_fp(scores):
    multiplier = np.array([1, .5, 1.25, 1.5, 2, 2, -.5, 1.5, 3])
    return np.dot(multiplier, scores)

for id in tqdm(test.draft.api_id):
    try:
        data = pd.read_csv(f'data/NBA_API/stats/2021/{id}.csv')
        data = data.drop(['pos', 'comment'], axis=1).dropna()

        data['min'] = data['min'].str.split(':').apply(lambda x: float(f"{x[0]}.{str((int(x[1])/60)).split('.')[-1]}"))
        factors = pd.DataFrame(data.mean().rename(id)).T.reset_index()
        factors = factors.rename({'index' : 'api_id'}, axis = 1)
        factors['dd'] = ((data[['points', 'totReb', 'assists', 'steals', 'blocks']] > 10).sum(axis=1) >= 2).astype(int).mean()
        factors['td'] = ((data[['points', 'totReb', 'assists', 'steals', 'blocks']] > 10).sum(axis=1) >= 3).astype(int).mean()

        fp_cols = ['points', 'tpm', 'totReb', 'assists', 'steals', 'blocks', 'turnovers', 'dd', 'td']
        factors['fp'] = calc_fp(np.array(factors[fp_cols]).reshape(-1, 1))

        df_factors.append(factors)
    except Exception as e:
        print(f'failed for player id: {id}, Error: {e}')
        continue
    # test.load_player_stats(params = {'id': id, 'season': 2021})
df_factors = pd.concat(df_factors)
df_factors = pd.merge(test.draft.reset_index(), df_factors, left_on = 'api_id', right_on = 'api_id').rename({'index': 'Name'}, axis = 1)

  0%|          | 0/138 [00:00<?, ?it/s]

failed for player id: 45, Error: list index out of range
failed for player id: 383, Error: Can only use .str accessor with string values!
failed for player id: 327, Error: list index out of range
failed for player id: 459, Error: list index out of range
failed for player id: 317, Error: list index out of range
failed for player id: 998, Error: list index out of range
failed for player id: 403, Error: list index out of range
failed for player id: 1227, Error: [Errno 2] No such file or directory: 'data/NBA_API/stats/2021/1227.csv'
failed for player id: 3428, Error: [Errno 2] No such file or directory: 'data/NBA_API/stats/2021/3428.csv'
failed for player id: 2242, Error: list index out of range
failed for player id: 207, Error: list index out of range
failed for player id: 468, Error: Can only use .str accessor with string values!
failed for player id: 3453, Error: [Errno 2] No such file or directory: 'data/NBA_API/stats/2021/3453.csv'
failed for player id: 3435, Error: [Errno 2] No such 

In [5]:
df_factors.head()

,Name,Position,ID,Time,Salary,Team,Opp,api_id,points,min,...,totReb,assists,pFouls,steals,turnovers,blocks,plusMinus,dd,td,fp
0,nikola jokic,C,830650,2022-10-11 01:00:00+00:00,10100.0,DEN,PHX,279,26.404762,32.617063,...,13.392857,7.595238,2.607143,1.428571,3.702381,0.869048,4.869048,0.761905,0.119048,59.431548
1,joel embiid,C,794508,2022-10-10 23:00:00+00:00,9900.0,PHI,CLE,159,29.101266,33.744726,...,11.265823,3.911392,2.721519,1.012658,3.088608,1.291139,4.835443,0.556962,0.000000,53.601266
2,james harden,PG/SG,395388,2022-10-10 23:00:00+00:00,9600.0,PHI,CLE,216,20.609756,36.346138,...,7.060976,9.658537,2.292683,1.134146,4.231707,0.536585,2.317073,0.426829,0.048780,47.039634
3,lamelo ball,PG,1257420,2022-10-10 23:00:00+00:00,9200.0,CHA,WAS,2566,19.548780,31.268089,...,6.439024,7.365854,3.060976,1.548780,3.146341,0.365854,0.731707,0.231707,0.024390,42.737805
4,devin booker,PG/SG,845564,2022-10-11 01:00:00+00:00,9100.0,PHX,DEN,64,25.204819,33.230924,...,4.795181,4.530120,2.518072,1.048193,2.433735,0.361446,5.554217,0.024096,0.000000,40.933735


In [6]:
df_optimize = df_factors[['Name', 'Position', 'Salary', 'fp']]
df_optimize.Position = df_optimize.Position.str.split('/')

positions = ['PG', 'SG', 'SF', 'PF', 'C']
for pos in positions:
    df_optimize[pos] = df_optimize['Position'].apply(lambda x: 1 if pos in x else 0)

df_optimize['G'] = df_optimize.Position.apply(lambda x: 1 if ('PG' in x or 'SG' in x) else 0 )
df_optimize['F'] = df_optimize.Position.apply(lambda x: 1 if ('SF' in x or 'PF' in x) else 0 )
df_optimize['UTIL'] = 1


In [7]:
df_optimize

,Name,Position,Salary,fp,PG,SG,SF,PF,C,G,F,UTIL
0,nikola jokic,[C],10100.0,59.431548,0,0,0,0,1,0,0,1
1,joel embiid,[C],9900.0,53.601266,0,0,0,0,1,0,0,1
2,james harden,"[PG, SG]",9600.0,47.039634,1,1,0,0,0,1,0,1
3,lamelo ball,[PG],9200.0,42.737805,1,0,0,0,0,1,0,1
4,devin booker,"[PG, SG]",9100.0,40.933735,1,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
99,mamadi diakite,[PF],3000.0,11.294118,0,0,0,1,0,0,1,1
100,nate hinton,[SF],3000.0,1.525000,0,0,1,0,0,0,1,1
101,josh okogie,"[SG, SF]",3000.0,4.640625,0,1,1,0,0,1,1,1
102,ish wainright,[PF],3000.0,3.615385,0,0,0,1,0,0,1,1


In [10]:
from pulp import *

model = LpProblem("daily_fantasy", LpMaximize)

# variable_names = list(df_optimize.columns[-8:])
n_players = df_optimize.shape[0]
n_positions = 8

variable_names = [str(i)+str(j) for j in range(1, n_players+1) for i in range(1, n_positions+1)]
#print(variable_names)

salary = np.array(df_optimize.Salary)
fpoints = np.array(df_optimize.fp).reshape(-1,1)



dv = LpVariable.matrix('X', variable_names, cat = "integer", lowBound=0, upBound=1)
allocation = np.array(dv).reshape(n_players, n_positions)

print(allocation)

[[X_11 X_21 X_31 X_41 X_51 X_61 X_71 X_81]
 [X_12 X_22 X_32 X_42 X_52 X_62 X_72 X_82]
 [X_13 X_23 X_33 X_43 X_53 X_63 X_73 X_83]
 [X_14 X_24 X_34 X_44 X_54 X_64 X_74 X_84]
 [X_15 X_25 X_35 X_45 X_55 X_65 X_75 X_85]
 [X_16 X_26 X_36 X_46 X_56 X_66 X_76 X_86]
 [X_17 X_27 X_37 X_47 X_57 X_67 X_77 X_87]
 [X_18 X_28 X_38 X_48 X_58 X_68 X_78 X_88]
 [X_19 X_29 X_39 X_49 X_59 X_69 X_79 X_89]
 [X_110 X_210 X_310 X_410 X_510 X_610 X_710 X_810]
 [X_111 X_211 X_311 X_411 X_511 X_611 X_711 X_811]
 [X_112 X_212 X_312 X_412 X_512 X_612 X_712 X_812]
 [X_113 X_213 X_313 X_413 X_513 X_613 X_713 X_813]
 [X_114 X_214 X_314 X_414 X_514 X_614 X_714 X_814]
 [X_115 X_215 X_315 X_415 X_515 X_615 X_715 X_815]
 [X_116 X_216 X_316 X_416 X_516 X_616 X_716 X_816]
 [X_117 X_217 X_317 X_417 X_517 X_617 X_717 X_817]
 [X_118 X_218 X_318 X_418 X_518 X_618 X_718 X_818]
 [X_119 X_219 X_319 X_419 X_519 X_619 X_719 X_819]
 [X_120 X_220 X_320 X_420 X_520 X_620 X_720 X_820]
 [X_121 X_221 X_321 X_421 X_521 X_621 X_721 X_821]
 

In [580]:
obj_func = lpDot(allocation.T, fpoints)